<a href="https://colab.research.google.com/github/ambicapvs/ambica_info5731_spring2021/blob/main/In_class_exercise_09_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [2]:
#loading train and test data
import pandas as pd
train_data = open("/content/stsa-train.txt").read()
train_labels, train_text = [], []
test_labels, test_text = [], []
for i, line in enumerate(train_data.split("\n")):
    content = line.split(" ")
    train_labels.append(content[0])
    train_text.append(" ".join(content[1:]))
train_dataframe = pd.DataFrame (list(zip(train_text, train_labels)) , columns = ['text', 'label'])
train_dataframe.head(10)

,text,label
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
5,campanella gets the tone just right -- funny i...,1
6,a fan film that for the uninitiated plays bett...,0
7,"béart and berling are both superb , while hupp...",1
8,"a little less extreme than in the past , with ...",0
9,the film is strictly routine .,0


In [3]:
# test data
test_data = open("/content/stsa-test.txt").read()
test_labels, test_text = [], []
for i, line in enumerate(test_data.split("\n")):
    content = line.split(" ")
    test_labels.append(content[0])
    test_text.append(" ".join(content[1:]))
# create a dataframe using texts and lables
test_dataframe = pd.DataFrame (list(zip(test_text, test_labels)) , columns = ['text', 'label'])
test_dataframe.head(10)


,text,label
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1
5,take care of my cat offers a refreshingly diff...,1
6,"acting , particularly by tambor , almost makes...",0
7,the movie exists for its soccer action and its...,1
8,arnold 's jump from little screen to big will ...,0
9,if this holiday movie is supposed to be a gift...,0


In [4]:
#Imports
from textblob import Word
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from textblob import TextBlob
import requests
from nltk.stem import PorterStemmer

#data cleaning
def DataCleaning(dataframe):
  dataframe["punctuationremoval"] = dataframe["text"].str.replace('[^\w\s]','')
  #remove special chars
  dataframe['special_chars'] = dataframe["punctuationremoval"].str.replace('[^A-Za-z0-9 ]+','')
  #remove Numbers
  dataframe['nums_removed'] = dataframe["special_chars"].str.replace('[^A-Za-z ]+','')
  #extract stopwords
  data = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
  a = data.text
  stop = a.split()
  #stopwords
  dataframe['stopwords'] = dataframe["nums_removed"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  #lowercase
  dataframe["lower_case"] = dataframe['stopwords'].apply(lambda x: " ".join(x.lower() for x in x.split()))
  #tokenizing data for next step - stemming
  dataframe["tokenization"] = dataframe["lower_case"].apply(lambda x: TextBlob(x).words)
  #stemming
  st = PorterStemmer()
  dataframe["stemming"]= dataframe["tokenization"].apply(lambda x: " ".join([st.stem(word) for word in x]))
  #Lemmatization
  dataframe["lemmatization"] = dataframe["stemming"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
  return dataframe


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
traindf = DataCleaning(train_dataframe)
traindf.head(10)

,text,label,punctuationremoval,special_chars,nums_removed,stopwords,lower_case,tokenization,stemming,lemmatization
0,"a stirring , funny and finally transporting re...",1,a stirring funny and finally transporting rei...,a stirring funny and finally transporting rei...,a stirring funny and finally transporting rei...,stirring funny finally transporting reimaginin...,stirring funny finally transporting reimaginin...,"[stirring, funny, finally, transporting, reima...",stir funni final transport reimagin beauti bea...,stir funni final transport reimagin beauti bea...
1,apparently reassembled from the cutting-room f...,0,apparently reassembled from the cuttingroom fl...,apparently reassembled from the cuttingroom fl...,apparently reassembled from the cuttingroom fl...,apparently reassembled cuttingroom floor given...,apparently reassembled cuttingroom floor given...,"[apparently, reassembled, cuttingroom, floor, ...",appar reassembl cuttingroom floor given daytim...,appar reassembl cuttingroom floor given daytim...
2,they presume their audience wo n't sit still f...,0,they presume their audience wo nt sit still fo...,they presume their audience wo nt sit still fo...,they presume their audience wo nt sit still fo...,presume audience wo nt sit still sociology les...,presume audience wo nt sit still sociology les...,"[presume, audience, wo, nt, sit, still, sociol...",presum audienc wo nt sit still sociolog lesson...,presum audienc wo nt sit still sociolog lesson...
3,this is a visually stunning rumination on love...,1,this is a visually stunning rumination on love...,this is a visually stunning rumination on love...,this is a visually stunning rumination on love...,visually stunning rumination love memory histo...,visually stunning rumination love memory histo...,"[visually, stunning, rumination, love, memory,...",visual stun rumin love memori histori war art ...,visual stun rumin love memori histori war art ...
4,jonathan parker 's bartleby should have been t...,1,jonathan parker s bartleby should have been th...,jonathan parker s bartleby should have been th...,jonathan parker s bartleby should have been th...,jonathan parker bartleby beallendall modernoff...,jonathan parker bartleby beallendall modernoff...,"[jonathan, parker, bartleby, beallendall, mode...",jonathan parker bartlebi beallendal modernoffi...,jonathan parker bartlebi beallendal modernoffi...
5,campanella gets the tone just right -- funny i...,1,campanella gets the tone just right funny in ...,campanella gets the tone just right funny in ...,campanella gets the tone just right funny in ...,campanella gets tone right funny middle sad mi...,campanella gets tone right funny middle sad mi...,"[campanella, gets, tone, right, funny, middle,...",campanella get tone right funni middl sad midd...,campanella get tone right funni middl sad midd...
6,a fan film that for the uninitiated plays bett...,0,a fan film that for the uninitiated plays bett...,a fan film that for the uninitiated plays bett...,a fan film that for the uninitiated plays bett...,fan film uninitiated plays better video sound ...,fan film uninitiated plays better video sound ...,"[fan, film, uninitiated, plays, better, video,...",fan film uniniti play better video sound turn,fan film uniniti play better video sound turn
7,"béart and berling are both superb , while hupp...",1,béart and berling are both superb while huppe...,bart and berling are both superb while hupper...,bart and berling are both superb while hupper...,bart berling superb huppert magnificent,bart berling superb huppert magnificent,"[bart, berling, superb, huppert, magnificent]",bart berl superb huppert magnific,bart berl superb huppert magnific
8,"a little less extreme than in the past , with ...",0,a little less extreme than in the past with l...,a little less extreme than in the past with l...,a little less extreme than in the past with l...,little less extreme past longer exposition seq...,little less extreme past longer exposition seq...,"[l

In [5]:
testdf = DataCleaning(test_dataframe)
testdf.head(10)

,text,label,punctuationremoval,special_chars,nums_removed,stopwords,lower_case,tokenization,stemming,lemmatization
0,"no movement , no yuks , not much of anything .",0,no movement no yuks not much of anything,no movement no yuks not much of anything,no movement no yuks not much of anything,movement yuks much anything,movement yuks much anything,"[movement, yuks, much, anything]",movement yuk much anyth,movement yuk much anyth
1,"a gob of drivel so sickly sweet , even the eag...",0,a gob of drivel so sickly sweet even the eage...,a gob of drivel so sickly sweet even the eage...,a gob of drivel so sickly sweet even the eage...,gob drivel sickly sweet even eager consumers m...,gob drivel sickly sweet even eager consumers m...,"[gob, drivel, sickly, sweet, even, eager, cons...",gob drivel sickli sweet even eager consum moor...,gob drivel sickli sweet even eager consum moor...
2,"gangs of new york is an unapologetic mess , wh...",0,gangs of new york is an unapologetic mess who...,gangs of new york is an unapologetic mess who...,gangs of new york is an unapologetic mess who...,gangs new york unapologetic mess whose saving ...,gangs new york unapologetic mess whose saving ...,"[gangs, new, york, unapologetic, mess, whose, ...",gang new york unapologet mess whose save grace...,gang new york unapologet mess whose save grace...
3,"we never really feel involved with the story ,...",0,we never really feel involved with the story ...,we never really feel involved with the story ...,we never really feel involved with the story ...,never really feel involved story ideas remain ...,never really feel involved story ideas remain ...,"[never, really, feel, involved, story, ideas, ...",never realli feel involv stori idea remain abs...,never realli feel involv stori idea remain abs...
4,this is one of polanski 's best films .,1,this is one of polanski s best films,this is one of polanski s best films,this is one of polanski s best films,one polanski best films,one polanski best films,"[one, polanski, best, films]",one polanski best film,one polanski best film
5,take care of my cat offers a refreshingly diff...,1,take care of my cat offers a refreshingly diff...,take care of my cat offers a refreshingly diff...,take care of my cat offers a refreshingly diff...,take care cat offers refreshingly different sl...,take care cat offers refreshingly different sl...,"[take, care, cat, offers, refreshingly, differ...",take care cat offer refreshingli differ slice ...,take care cat offer refreshingli differ slice ...
6,"acting , particularly by tambor , almost makes...",0,acting particularly by tambor almost makes ...,acting particularly by tambor almost makes ...,acting particularly by tambor almost makes ...,acting particularly tambor almost makes never ...,acting particularly tambor almost makes never ...,"[acting, particularly, tambor, almost, makes, ...",act particularli tambor almost make never wort...,act particularli tambor almost make never wort...
7,the movie exists for its soccer action and its...,1,the movie exists for its soccer action and its...,the movie exists for its soccer action and its...,the movie exists for its soccer action and its...,movie exists soccer action fine acting,movie exists soccer action fine acting,"[movie, exists, soccer, action, fine, acting]",movi exist soccer action fine act,movi exist soccer action fine act
8,arnold 's jump from little screen to big will ...,0,arnold s jump from little screen to big will l...,arnold s jump from little screen to big will l...,arnold s jump from little screen to big will l...,arnold jump little screen big leave frowns faces,arnold jump little screen big leave frowns faces,"[arnold, jump, little, screen, big, leave, fro...",arnold jump littl screen big leav frown face,arnold jump littl screen big leav frown face
9,if this holiday movie is supposed to be a gift...,0,if this holiday movie is supposed to be a gift...,if this holiday movie is supposed to be a gift...,if this holiday movie is

In [7]:
#TF-IDF vectorization
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
tfidf_vect = TfidfVectorizer(analyzer='word')
tfidf_vect.fit(traindf['lemmatization'])
x_tfidf =  tfidf_vect.transform(traindf['lemmatization'])
vect_test = TfidfVectorizer(analyzer='word', vocabulary = tfidf_vect.vocabulary_)
vect_test.fit(testdf['lemmatization'])
test_x = vect_test.transform(testdf['lemmatization'])
test_y = testdf['label']



In [8]:
#split the dataset into training and validation datasets 
from sklearn import model_selection, preprocessing
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(x_tfidf, traindf['label'], test_size=0.2)


In [9]:
#Function for Cross Validation Score
from sklearn.model_selection import cross_val_score, KFold
from sklearn import metrics
def get_cross_score(modelName, x, y):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7,shuffle=True)
  cross_score = cross_val_score(modelName, x, y, cv=kfold).mean()
  return cross_score
#Function for various scores - accuracy, precision, recall and F1
def get_metrics(predictions, test_data_y):
  accuracy = metrics.accuracy_score(predictions, test_data_y)
  precision = metrics.precision_score(predictions, test_data_y, pos_label='positive', average='micro')
  recall = metrics.recall_score(predictions, test_data_y, pos_label='positive', average='micro')
  f1 = metrics.f1_score(predictions, test_data_y, pos_label='positive', average='micro')
  return accuracy, precision, recall, f1

In [10]:
#Multinomial NB Model
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def Naive_bayes_model(x1, y1, x2, y2):
  nb_model = naive_bayes.MultinomialNB()
  nb_model.fit(x1, y1)
  nb_predicts = nb_model.predict(x2)
  accuracy, precision, recall, f1 = get_metrics(nb_predicts, y2)
  nb_score = get_cross_score(nb_model, x2, y2)
  return accuracy, precision, recall, f1, nb_score

accuracy_nb, precision_nb, recall_nb, f1_nb, nb_score = Naive_bayes_model(train_x, train_y, valid_x, valid_y)
print("Printing results for Naive Bayes Model:\n")
print("Training Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_nb, precision_nb, recall_nb, f1_nb, nb_score))
accuracy_nbt, precision_nbt, recall_nbt, f1_nbt, nb_scoret = Naive_bayes_model(train_x, train_y, test_x, test_y)
print("\nTest Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_nbt, precision_nbt, recall_nbt, f1_nbt, nb_scoret))


Printing results for Naive Bayes Model:

Training Data:
Accuracy : 0.7812274368231047
Precision : 0.7812274368231047
Recall : 0.7812274368231047
F1 : 0.7812274368231046
Cross validation Score : 0.7126159941611927

Test Data:
Accuracy : 0.7854006586169044
Precision : 0.7854006586169044
Recall : 0.7854006586169044
F1 : 0.7854006586169044
Cross validation Score : 0.7365219479973579


**SVM**

In [11]:
#SVM model

def SVM_model_Fun(x1, y1, x2, y2):
  svm_model = svm.SVC()
  svm_model.fit(x1, y1)
  svm_predicts = svm_model.predict(x2)
  accuracy, precision, recall, f1 = get_metrics(svm_predicts, y2)
  svm_score = get_cross_score(svm_model, x2, y2)
  return accuracy, precision, recall, f1, svm_score

accuracy_svm, precision_svm, recall_svm, f1_svm, svm_score = SVM_model_Fun(train_x, train_y, valid_x, valid_y)
print("Printing results for SVM Model:\n")
print("Training Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_svm, precision_svm, recall_svm, f1_svm, svm_score))
accuracy_svmt, precision_svmt, recall_svmt, f1_svmt, svm_scoret = SVM_model_Fun(train_x, train_y, test_x, test_y)
print("\nTest Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_svmt, precision_svmt, recall_svmt, f1_svmt, svm_scoret))




Printing results for SVM Model:

Training Data:
Accuracy : 0.7805054151624549
Precision : 0.7805054151624549
Recall : 0.7805054151624549
F1 : 0.7805054151624549
Cross validation Score : 0.7227400688145136

Test Data:
Accuracy : 0.7930845225027442
Precision : 0.7930845225027442
Recall : 0.7930845225027442
F1 : 0.7930845225027442
Cross validation Score : 0.7425689065033328


**KNN**

In [12]:
#KNN model
from sklearn.neighbors import KNeighborsClassifier
def KNN_model_Fun(x1, y1, x2, y2):
  knn_model = KNeighborsClassifier(n_neighbors = 15)
  knn_model.fit(x1, y1)
  knn_predicts = knn_model.predict(x2)
  accuracy, precision, recall, f1 = get_metrics(knn_predicts, y2)
  knn_score = get_cross_score(knn_model, x2, y2)
  return accuracy, precision, recall, f1, knn_score

accuracy_knn, precision_knn, recall_knn, f1_knn, knn_score = KNN_model_Fun(train_x, train_y, valid_x, valid_y)
print("Printing results for KNN Model:\n")
print("Training Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_knn, precision_knn, recall_knn, f1_knn, knn_score))
accuracy_knnt, precision_knnt, recall_knnt, f1_knnt, knn_scoret = KNN_model_Fun(train_x, train_y, test_x, test_y)
print("\nTest Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_knnt, precision_knnt, recall_knnt, f1_knnt, knn_scoret))

Printing results for KNN Model:

Training Data:
Accuracy : 0.7371841155234657
Precision : 0.7371841155234657
Recall : 0.7371841155234657
F1 : 0.7371841155234657
Cross validation Score : 0.6764883745177771

Test Data:
Accuracy : 0.7354555433589463
Precision : 0.7354555433589463
Recall : 0.7354555433589463
F1 : 0.7354555433589463
Cross validation Score : 0.6465381612922597


**DECISION TREE**

In [13]:
# decision tree

from sklearn.tree import DecisionTreeClassifier
def decision_Tree(x1, y1, x2, y2):
  dt_model = DecisionTreeClassifier()
  dt_model.fit(x1, y1)
  dt_predicts = dt_model.predict(x2)
  accuracy, precision, recall, f1 = get_metrics(dt_predicts, y2)
  dt_score = get_cross_score(dt_model, x2, y2)
  return accuracy, precision, recall, f1, dt_score

accuracy_dt, precision_dt, recall_dt, f1_dt, dt_score = decision_Tree(train_x, train_y, valid_x, valid_y)
print("Printing results for Decision Tree:\n")
print("Training Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_dt, precision_dt, recall_dt, f1_dt, dt_score))
accuracy_dtt, precision_dtt, recall_dtt, f1_dtt, dt_scoret = decision_Tree(train_x, train_y, test_x, test_y)
print("\nTest Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_dtt, precision_dtt, recall_dtt, f1_dtt, dt_scoret))

Printing results for Decision Tree:

Training Data:
Accuracy : 0.6750902527075813
Precision : 0.6750902527075813
Recall : 0.6750902527075813
F1 : 0.6750902527075813
Cross validation Score : 0.5371806902304244

Test Data:
Accuracy : 0.677277716794731
Precision : 0.677277716794731
Recall : 0.677277716794731
F1 : 0.677277716794731
Cross validation Score : 0.650354290518225


**Random Forest**

In [14]:
#random Forest

from sklearn.ensemble import RandomForestClassifier
def random_Forest(x1, y1, x2, y2):
  rf_model = RandomForestClassifier()
  rf_model.fit(x1, y1)
  rf_predicts = rf_model.predict(x2)
  accuracy, precision, recall, f1 = get_metrics(rf_predicts, y2)
  rf_score = get_cross_score(rf_model, x2, y2)
  return accuracy, precision, recall, f1, rf_score

accuracy_rf, precision_rf, recall_rf, f1_rf, rf_score = random_Forest(train_x, train_y, valid_x, valid_y)
print("Printing results for Random Forest:\n")
print("Training Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_rf, precision_rf, recall_rf, f1_rf, rf_score))
accuracy_rft, precision_rft, recall_rft, f1_rft, rft_score = random_Forest(train_x, train_y, test_x, test_y)
print("\nTest Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_rft, precision_rft, recall_rft, f1_rft, rft_score))

Printing results for Random Forest:

Training Data:
Accuracy : 0.7537906137184116
Precision : 0.7537906137184116
Recall : 0.7537906137184116
F1 : 0.7537906137184115
Cross validation Score : 0.6851736002502348

Test Data:
Accuracy : 0.7557628979143798
Precision : 0.7557628979143798
Recall : 0.7557628979143798
F1 : 0.7557628979143798
Cross validation Score : 0.6833123160992014


**XG Boost**

In [15]:
#XG Boost

from xgboost import XGBClassifier
def XGBoostmodel(x1, y1, x2, y2):
  xg_model = XGBClassifier()
  xg_model.fit(x1, y1)
  xg_predicts = xg_model.predict(x2)
  accuracy, precision, recall, f1 = get_metrics(xg_predicts, y2)
  xg_score = get_cross_score(XGBClassifier(), valid_x, valid_y)
  return accuracy, precision, recall, f1, xg_score

accuracy_xg, precision_xg, recall_xg, f1_xg, xg_score = XGBoostmodel(train_x, train_y, valid_x, valid_y)
print("Printing results for XG Boost:\n")
print("Training Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_xg, precision_xg, recall_xg, f1_xg, xg_score))
accuracy_xgt, precision_xgt, recall_xgt, f1_xgt, xgt_score = XGBoostmodel(train_x, train_y, test_x, test_y)
print("\nTest Data:\nAccuracy : {0}\nPrecision : {1}\nRecall : {2}\nF1 : {3}\nCross validation Score : {4}".format(accuracy_xgt, precision_xgt, recall_xgt, f1_xgt, xgt_score))

Printing results for XG Boost:

Training Data:
Accuracy : 0.644043321299639
Precision : 0.644043321299639
Recall : 0.644043321299639
F1 : 0.644043321299639
Cross validation Score : 0.5942498175372745

Test Data:
Accuracy : 0.637211855104281
Precision : 0.637211855104281
Recall : 0.637211855104281
F1 : 0.637211855104281
Cross validation Score : 0.5942498175372745


**Comparing Accuracies**

In [16]:
print("Accuracies with Training data:")
print("\nNaive Baiyes : {0:.2f}% \nSVM model: {1:.2f}% \nKNN Model: {2:.2f}%  \nDecision Tree: {3:.2f}% \nRandom Forest: {4:.2f}% \nXG Boost: {5:.2f}% ".format(accuracy_nb * 100, accuracy_svm * 100, accuracy_knn * 100, accuracy_dt * 100, accuracy_rf * 100, accuracy_xg * 100 ))

Accuracies with Training data:

Naive Baiyes : 78.12% 
SVM model: 78.05% 
KNN Model: 73.72%  
Decision Tree: 67.51% 
Random Forest: 75.38% 
XG Boost: 64.40% 


In [17]:
print("Accuracies with Test data:")
print("\nNaive Baiyes : {0:.2f}% \nSVM model: {1:.2f}% \nKNN Model: {2:.2f}%  \nDecision Tree: {3:.2f}% \nRandom Forest: {4:.2f}% \nXG Boost: {5:.2f}% ".format(accuracy_nbt * 100, accuracy_svmt * 100, accuracy_knnt * 100, accuracy_dtt * 100, accuracy_rft * 100, accuracy_xgt * 100 ))

Accuracies with Test data:

Naive Baiyes : 78.54% 
SVM model: 79.31% 
KNN Model: 73.55%  
Decision Tree: 67.73% 
Random Forest: 75.58% 
XG Boost: 63.72% 
